<a href="https://colab.research.google.com/github/ha191180/practice-BERT/blob/main/bert_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [2]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0


import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

     |████████████████████████████████| 2.1 MB 7.3 MB/s 
     |████████████████████████████████| 486 kB 65.8 MB/s 
     |████████████████████████████████| 13.4 MB 200 kB/s 
     |████████████████████████████████| 895 kB 57.1 MB/s 
     |████████████████████████████████| 3.3 MB 56.3 MB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=512a62e4b47fb46bf4a722caa61b9f155f587278ec5ceb5cac24efe125cd08e1
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
Successfully built ipadic


# Tokenizer and model

In [3]:
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
bert_mlm = bert_mlm.cuda()

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
text = '今日は[MASK]へ行く。'
tokens = tokenizer.tokenize(text)
print(tokens)

['今日', 'は', '[MASK]', 'へ', '行く', '。']


# Prediction

In [12]:
input_ids = tokenizer.encode(text, return_tensors='pt')
input_ids = input_ids.cuda()

with torch.no_grad():
  output = bert_mlm(input_ids=input_ids)
scores = output.logits

mask_position = input_ids[0].tolist().index(4)
topk = scores[0, mask_position].topk(10)
ids_topk = topk.indices
tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk)
scores_topk = topk.values.cpu().numpy()

text_topk = []
for token in tokens_topk:
  token = token.replace('##', '')
  text_topk.append(text.replace('[MASK]', token, 1))

for x in range(len(scores_topk)):
  print(text_topk[x], scores_topk[x] )

今日は東京へ行く。 9.178562
今日はハワイへ行く。 9.1459465
今日は学校へ行く。 8.92324
今日はニューヨークへ行く。 8.838818
今日はどこへ行く。 8.319209
今日は空港へ行く。 8.180514
今日はアメリカへ行く。 7.917594
今日は病院へ行く。 7.8333464
今日はそこへ行く。 7.826692
今日はロンドンへ行く。 7.8070273
